# 검증 세트
- 테스트 세트를 사용하지 않으면 모델이 과대적합인지 과소적합인지 판단하기 어렵다. 테스트 세트를 사용하지 않고 이를 측정하는 간단한 방법은 훈련 세트를 또 나누는 것! 이 데이터를 검증세트(validation set)라고 한다.

In [2]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [3]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

- 훈련 세트의 입력 데이터와 타깃 데이터를 train_input과 train_target 배열에 저장

In [4]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

- 그다음 train_input과 train_target을 다시 train_test_split() 함수에 넣어 훈련 세트 sub_input, sub_target과
검증 세트 val_input, val_target을 만듭니다. 여기에서도 test_size 매개변수를 0.2로 지정하여 train_input의 약 20%를 val_input으로 만든다.
- 단순히 train_test_split() 함수를 2번 적용해서 훈련 세트와 검증세트로 나눠준것

In [5]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [7]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [8]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


# 교차 검증
- 검증 세트를 만드느라 훈련 세트가 줄었다. 보통 많은 데이터를 훈련에 사용할수록 좋은 모델이 만들어집니다. 그렇다고 검증 세트를 너무 조금 떼어 놓으면 검증 점수가 들쭉날쭉하고 불안정할 것. 이럴 때 교차 검증(cross validation)을 이용하면 안정적인 검증 점수를 얻고 훈련에 더 많은 데이터를 사용할 수 있습니다.

In [9]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00799775, 0.00797653, 0.0079782 , 0.0079782 , 0.00701046]), 'score_time': array([0.00098038, 0.        , 0.00099778, 0.        , 0.        ]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


- 이 함수는 fit_time, score_time, test_score키를 가진 딕셔너리를 반환. 처음 2개의 키는 각각 모델을 훈련하는 시간과 검증하는 시간을 의미합니다. 각 키마다 5개의 숫자가 담겨 있습니다. cross_validate()함수는 기본적으로 5-폴드 교차 검증을 수행.
cv 매개변수에서 폴드 수를 바꿀 수도 있습니다.
- 교차 검증의 최종 점수는 test_score키에 담긴 5개의 점수를 평균하여 얻을 수 있다.
이름은 test_score지만 검증 폴드의 점수이니 혼동하지 말자.

In [10]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


# 분할기를 사용한 교차 검증
- 사이킷런의 분할기는 교차 검증에서 폴드를 어떻게 나눌지 결정해 준다. cross_validate()함수는 기본적으로 회귀 모델일 경우KFold분할기를 사용하고 분류 모델일 경우 타깃 클래스를 골고루 나누기 위해 StratifiedKFold를 사용합니다. 즉 앞서 수행한 교차 검증은 다음 코드와 동일

In [12]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [13]:
# 만약 훈련 세트를 섞은 후 10-폴드 교차 검증을 수행하려면..
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


# 그리드 서치

In [14]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [16]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params,n_jobs=-1) #-1은 가능한 모든코어를 다사용해서 모델훈련
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [17]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [18]:
print(gs.best_params_) # 5개중에 0.0001이 가장 좋더라

{'min_impurity_decrease': 0.0001}


In [19]:
print(gs.cv_results_['mean_test_score']) 

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [20]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


1. 먼저 탐색할 매개변수를 지정
2. 그다음 훈련 세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾습니다. 이 조합은 그리드 서치 객체에 저장됩니다.
3. 그리드 서치는 최상의 매개변수에서 (교차 검증에 사용한 훈련 세트가 아니라) 전체 훈련 세트를 사용해 최종 모델을 훈련합니다. 이 모델도 그리드 서치 객체에 저장됩니다.

# 확률 분포 선택
- 싸이파이는 파이썬의 핵심 라이브러리 중 하나입니다. 적분, 보간, 선형 대수, 확률 등을 포함한 수치 계산 전용 라이브러리입니다.
사이킷런은 넘파이와 싸이파이 기능을 많이 사용합니다.

In [22]:
from scipy.stats import uniform, randint
# uniform과 randint 클래스 모두 주어지 범위에서 고르게 값을 뽑는다. '균등 분포에서 샘플링한다'

In [23]:
rgen = randint(0, 10)
rgen.rvs(10)

array([0, 6, 7, 3, 6, 2, 3, 3, 1, 7], dtype=int64)

In [24]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64),
 array([ 98, 114,  91, 106,  86, 105, 106, 107,  96,  91], dtype=int64))

In [25]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.26437018, 0.67831312, 0.26799616, 0.69917374, 0.83857579,
       0.47737054, 0.52116776, 0.04668258, 0.09220971, 0.28991798])

# 랜덤 서치

In [26]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001), 
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [27]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001B1BC3F6250>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001B1C0FAC410>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001B1BC3E5210>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001B1BC3F4850>},
                   random_state=42)

In [28]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [29]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [30]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86
